# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
from pyspark.sql.types import (StringType, IntegerType)


# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [2]:
spark = SparkSession \
    .builder \
    .master('local[4]') \
    .appName("Wrangling Data2") \
    .getOrCreate()

In [3]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [4]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [14]:
user_log.limit(10).toPandas().T

,0,1,2,3,4,5,6,7,8,9
artist,Showaddywaddy,Lily Allen,Cobra Starship Featuring Leighton Meester,Alex Smoke,None,None,Redman,Ulrich Schnauss,None,None
auth,Logged In,Logged In,Logged In,Logged In,Logged In,Logged In,Logged In,Logged In,Logged In,Logged In
firstName,Kenneth,Elizabeth,Vera,Sophee,Jordyn,Jordyn,Paige,Gabriel,Mason,Alexander
gender,M,F,F,F,F,F,F,M,M,M
itemInSession,112,7,6,8,0,1,0,2,2,0
lastName,Matthews,Chase,Blackwell,Barker,Jones,Jones,Hunter,Koch,Thomas,Short
length,232.933,195.239,196.205,405.995,NaN,NaN,154.54,402.938,NaN,NaN
level,paid,free,paid,paid,free,free,free,paid,paid,free
location,"Charlotte-Concord-Gastonia, NC-SC","Shreveport-Bossier City, LA","Racine, WI","San Luis Obispo-Paso Robles-Arroyo Grande, CA","Syracuse, NY","Syracuse, NY","Brownwood, TX","Panama City, FL","Indianapolis-Carmel-Anderson, IN","Green Bay, WI"
method,PUT,PUT,PUT,PUT,GET,GET,PUT,PUT,GET,GET


In [6]:
pages_visited = user_log.where(user_log.userId == '').select('page').distinct().toPandas()['page'].tolist()
all_pages = user_log.select('page').distinct().toPandas()['page'].tolist()
not_visited = set(all_pages) - set(pages_visited)
not_visited

{'Downgrade',
 'Error',
 'Logout',
 'NextSong',
 'Save Settings',
 'Settings',
 'Submit Downgrade',
 'Submit Upgrade',
 'Upgrade'}

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [7]:
user_log.where(F.col('userId') == '').groupBy(F.col('level')).count().collect()

[Row(level='free', count=193), Row(level='paid', count=143)]

# Question 3

How many female users do we have in the data set?

In [9]:
user_log.where(F.col('gender') == 'F').selectExpr('COUNT(DISTINCT(userId))').collect()

[Row(count(DISTINCT userId)=462)]

# Question 4

How many songs were played from the most played artist?

In [13]:
user_log.groupBy('artist').count().orderBy(F.desc('count')).show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|                null| 1653|
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|               Train|   28|
|          Lily Allen|   28|
|Barry Tuckwell/Ac...|   28|
|           Metallica|   27|
|          Nickelback|   27|
|           Daft Punk|   27|
|          Kanye West|   26|
+--------------------+-----+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [15]:
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

In [22]:
function = F.udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(F.desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, Window.currentRow)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(F.col('page'))) \
    .withColumn('period', F.sum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

